In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 1. Data Preparation
imdb = keras.datasets.imdb
(vocab_size) = 10000
(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words=vocab_size)

# Preprocess the data - Padding the arrays so they all have the same length
max_length = 256
train_data = keras.preprocessing.sequence.pad_sequences(train_data, value=0, padding='post', maxlen=max_length)
test_data = keras.preprocessing.sequence.pad_sequences(test_data, value=0, padding='post', maxlen=max_length)

# 2. Model Building
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, 16, input_length=max_length),
    keras.layers.GlobalAveragePooling1D(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 3. Training
x_val = train_data[:10000]
partial_x_train = train_data[10000:]
y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

history = model.fit(partial_x_train, partial_y_train, epochs=40, batch_size=512, validation_data=(x_val, y_val), verbose=1)

# 4. Evaluation
results = model.evaluate(test_data, test_labels, verbose=2)
print(results)




17464789/17464789 [==============================] - 1s 0us/step


Epoch 1/40


30/30 [==============================] - 1s 16ms/step - loss: 0.6921 - accuracy: 0.5633 - val_loss: 0.6904 - val_accuracy: 0.5865
Epoch 2/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6869 - accuracy: 0.6599 - val_loss: 0.6831 - val_accuracy: 0.7202
Epoch 3/40
30/30 [==============================] - 0s 13ms/step - loss: 0.6753 - accuracy: 0.7435 - val_loss: 0.6685 - val_accuracy: 0.7427
Epoch 4/40
30/30 [==============================] - 0s 13ms/step - loss: 0.6543 - accuracy: 0.7633 - val_loss: 0.6445 - val_accuracy: 0.7515
Epoch 5/40
30/30 [==============================] - 0s 12ms/step - loss: 0.6232 - accuracy: 0.7775 - val_loss: 0.6118 - val_accuracy: 0.7777
Epoch 6/40
30/30 [==============================] - 0s 12ms/step - loss: 0.5836 - accuracy: 0.8047 - val_loss: 0.5745 - val_accuracy: 0.7915
Epoch 7/40
30/30 [==============================] - 0s 13ms/step - loss: 0.5402 - ac

In [4]:
# Load the IMDb word index
word_index = imdb.get_word_index()

# Create a reverse word index
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

# Function to encode a given text into integers
def encode_text(text):
    # IMDb reserves the first 3 indices for special tokens
    encoded = [1]  # Start with index 1 for the start of the sequence
    for word in text.split():
        if word.lower() in word_index:
            encoded.append(word_index[word.lower()] + 3)  # Offset by 3 as IMDb datasets have reserved indices at the start
        else:
            encoded.append(2)  # Use 2 for unknown words
    return encoded


1641221/1641221 [==============================] - 0s 0us/step


In [12]:
# Function to preprocess new reviews
def preprocess_review(text):
    encoded = encode_text(text)
    padded = keras.preprocessing.sequence.pad_sequences([encoded], value=0, padding='post', maxlen=max_length)  # Use the same max_length as during training
    return padded

# Example review
new_review = "This movie was EXTREMELY BAD AND I HATED IT"

# Preprocess the review
preprocessed_review = preprocess_review(new_review)

# Making a prediction
prediction = model.predict(preprocessed_review)

print("Prediction (0 for negative, 1 for positive):", prediction[0])

# Interpreting the prediction
if prediction[0] > 0.5:
    print("Positive sentiment")
else:
    print("Negative sentiment")


1/1 [==============================] - 0s 22ms/step
Prediction (0 for negative, 1 for positive): [0.2968883]
Negative sentiment
